In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import tensorflow as tf
import numpy as np
import h5py, os, glob, re, itertools, datetime

In [0]:
train_dir = "/content/drive/My Drive/DL/DIV2K/train/"
valid_dir = "/content/drive/My Drive/DL/DIV2K/valid/"
hdf_dir = "/content/drive/My Drive/DL/DIV2K HDF/"
dir_type = { 0:"HR/", 1:"LR_bicub_X2/", 2:"LR_bicub_X3/", 3:"LR_bicub_X4/", 
               4:"LR_unkn_X2/", 5:"LR_unkn_X3/", 6:"LR_unkn_X4/" }

In [0]:
def get_hdf_dir(training=True, hsv=False):
    if hsv == False:
        cmodel = "RGB/"
    else: cmodel = "HSV/"
    if training == True:
        data = "train/"
    else: data = "valid/"
    return os.path.join(hdf_dir, cmodel, data)

def paths_gen(training=True):
    if training == True:
        directory = train_dir
    else: directory = valid_dir
    hr_paths = glob.glob(os.path.join(directory, dir_type[0]) + "*.png")
    for i in range(len(hr_paths)):
        yield hr_paths[i]
        
def path2image(img_path):
    img_raw = tf.read_file(img_path)
    img_dec = tf.image.decode_png(img_raw, channels=3)
    img = tf.image.convert_image_dtype(img_dec, tf.float32)
    return img

def patches_creating(img, patch_size=48):
    patchsize = [1, patch_size, patch_size, 1]
    patches = tf.extract_image_patches(img, patchsize, patchsize, [1, 1, 1, 1], 'VALID')
    return patches
    
def postpatching(patches, patch_size=48):
    _, R, C, _ = np.asarray(patches.shape)
    x2_patch_size = np.asarray([patch_size, patch_size])//2
    x4_patch_size = np.asarray([patch_size, patch_size])//4
    hr_patches_list = [] 
    lr_patches_x2_list = [] 
    lr_patches_x4_list = []
    patched_image_size = [tf.constant(R), tf.constant(C)]
    for r, c in itertools.product(*map(range, [R, C])):
        hr_patch = tf.reshape(patches[0, r, c, :], [patch_size, patch_size, 3])
        lr_patch_x2 = tf.image.resize_images(hr_patch, x2_patch_size, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        lr_patch_x4 = tf.image.resize_images(hr_patch, x4_patch_size, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        hr_patches_list.append(hr_patch)
        lr_patches_x2_list.append(lr_patch_x2)
        lr_patches_x4_list.append(lr_patch_x4)
    return hr_patches_list, lr_patches_x2_list, lr_patches_x4_list, patched_image_size

In [0]:
#if HSV = False => RGB = True
TRAINING = False
HSV = True
PATCH_SIZE = 48

#super-duper counter for save-name
I = 1

print("Start:", datetime.datetime.now().strftime("%m/%d %H:%M:%S"))
for img_path in paths_gen(training=TRAINING):
    tf.reset_default_graph()
    sess = tf.Session()
    with sess.as_default():
        img = sess.run(path2image(img_path))
        if HSV == True:
            img = sess.run(tf.image.rgb_to_hsv(img))
        img = tf.expand_dims(img, 0)
        patches = sess.run(patches_creating(img, patch_size=PATCH_SIZE))
        hr, lr_x2, lr_x4, patched_image_size = sess.run(postpatching(patches, patch_size=PATCH_SIZE))
        loop_hdf_dir = get_hdf_dir(training=TRAINING, hsv=HSV)
        with h5py.File(loop_hdf_dir + "img{}.hdf5".format(I), "w") as hdf:
            hdf.create_dataset("hr_img{}".format(I), data=np.asarray(hr))
            hdf.create_dataset("lr_x2_img{}".format(I), data=np.asarray(lr_x2))
            hdf.create_dataset("lr_x4_img{}".format(I), data=np.asarray(lr_x4))
            hdf.create_dataset("patched_res_img{}".format(I), data=np.asarray(patched_image_size))
        print("Image {} done, time: {}".format(re.split(r"/", img_path)[-1], datetime.datetime.now().strftime("%m/%d %H:%M:%S")))
        I += 1